## Day 27 Lecture 1 Assignment

In this assignment, we will learn statistical significance in linear models. We will use the google play store dataset loaded below and analyze the regression from this dataset.

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pylab
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
reviews = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/googleplaystore.csv')

In [0]:
reviews.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


We will predict app ratings using other features describing the app. To use these features, we must clean the data.

Start by creating dummy variables out of the type and content rating columns.

In [0]:
# answer below:
reviewsdm = pd.concat([reviews, pd.get_dummies(reviews['Type'], drop_first=True, prefix ='Type'), pd.get_dummies(reviews['Content Rating'], drop_first=True, prefix ='Content Rating')], axis = 1)
reviewsdm.Rating.interpolate(inplace=True)
reviewsdm.info()
reviewsdm.drop('Type', axis=1)
reviewsdm.drop('Content Rating', axis=1)

reviewsdm.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   App                          10841 non-null  object 
 1   Category                     10841 non-null  object 
 2   Rating                       10841 non-null  float64
 3   Reviews                      10841 non-null  object 
 4   Size                         10841 non-null  object 
 5   Installs                     10841 non-null  object 
 6   Type                         10840 non-null  object 
 7   Price                        10841 non-null  object 
 8   Content Rating               10840 non-null  object 
 9   Genres                       10841 non-null  object 
 10  Last Updated                 10841 non-null  object 
 11  Current Ver                  10833 non-null  object 
 12  Android Ver                  10838 non-null  object 
 13  Installsn       

Next, check for missing values and remove all rows containing missing values

In [0]:
# answer below:


reviewsclean = reviewsdm.drop(reviewsdm[reviewsdm['Price'] == 'Everyone'].index)

reviewsclean['Price']= pd.to_numeric(reviewsclean.Price.str.replace('$', ''))
reviewsclean['Reviews'] = pd.to_numeric(reviewsclean.Reviews)

['0' '0.99' '1.00' '1.04' '1.20' '1.26' '1.29' '1.49' '1.50' '1.59' '1.61'
 '1.70' '1.75' '1.76' '1.96' '1.97' '1.99' '10.00' '10.99' '109.99'
 '11.99' '12.99' '13.99' '14.00' '14.99' '15.46' '15.99' '154.99' '16.99'
 '17.99' '18.99' '19.40' '19.90' '19.99' '2.00' '2.49' '2.50' '2.56'
 '2.59' '2.60' '2.90' '2.95' '2.99' '200.00' '24.99' '25.99' '28.99'
 '29.99' '299.99' '3.02' '3.04' '3.08' '3.28' '3.49' '3.61' '3.88' '3.90'
 '3.95' '3.99' '30.99' '33.99' '37.99' '379.99' '389.99' '39.99' '394.99'
 '399.99' '4.29' '4.49' '4.59' '4.60' '4.77' '4.80' '4.84' '4.85' '4.99'
 '400.00' '46.99' '5.00' '5.49' '5.99' '6.49' '6.99' '7.49' '7.99' '74.99'
 '79.99' '8.49' '8.99' '89.99' '9.00' '9.99']


To simplify, we will remove the app, category, size, installs, genres, last updated, current ver, and android ver columns. 

In [0]:
# answer below:
reviewsdmd = reviewsclean.drop(['App', 'Category', 'Size', 'Installs', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver', 'Installsn', 'Type','Content Rating'],axis =1)
print(reviewsdmd.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10840 entries, 0 to 10840
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Rating                       10840 non-null  float64
 1   Reviews                      10840 non-null  int64  
 2   Price                        10840 non-null  float64
 3   Type_Free                    10840 non-null  uint8  
 4   Type_Paid                    10840 non-null  uint8  
 5   Content Rating_Everyone      10840 non-null  uint8  
 6   Content Rating_Everyone 10+  10840 non-null  uint8  
 7   Content Rating_Mature 17+    10840 non-null  uint8  
 8   Content Rating_Teen          10840 non-null  uint8  
 9   Content Rating_Unrated       10840 non-null  uint8  
dtypes: float64(2), int64(1), uint8(7)
memory usage: 412.9 KB
None


Next, check that all the columns are of numeric type and change the type of columns that are not numeric. If coercing to numeric causes missing values, remove those rows containing missing values from our dataset.

In [0]:
# answer below:
X = reviewsdmd.drop('Rating', axis = 1)
y = reviewsdmd['Rating']


Perform a train test split with 20% of the data in the test sample.

In [0]:
# answer below:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

Now generate a linear model using statsmodels or sklearn and produce a p value for each coefficient in the model. Analyze the results.

In [0]:
#answer below:
import statsmodels.api as sm
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Rating   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     9.256
Date:                Tue, 26 May 2020   Prob (F-statistic):           4.07e-14
Time:                        19:24:32   Log-Likelihood:                -8200.7
No. Observations:               10840   AIC:                         1.642e+04
Df Residuals:                   10830   BIC:                         1.649e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [0]:
import seaborn as sns
sns.scatter